## Perturbation on 500 Samples from train/seen set

#### ZC-CSA

In [1]:
import os
import time
import joblib
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# ──────────────────────────────────────────────────────────
# Numeric bounds & device
# ──────────────────────────────────────────────────────────
raw_upper_bound  = 50.0
raw_lower_bound  = -70.0
final_lower_bound = 0.0
final_upper_bound = 255.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_lower  = torch.tensor(raw_lower_bound,  dtype=torch.float32, device=device)
raw_upper  = torch.tensor(raw_upper_bound,  dtype=torch.float32, device=device)
final_lower = torch.tensor(final_lower_bound, dtype=torch.float32, device=device)
final_upper = torch.tensor(final_upper_bound, dtype=torch.float32, device=device)

img_shape = (28, 28)
dim = img_shape[0] * img_shape[1]

# ──────────────────────────────────────────────────────────
# CNN switch
# ──────────────────────────────────────────────────────────
# Flip this to True when you load a convolutional network that
# expects input shaped [N, C, H, W] rather than flattened.
CNN_MODE = False           # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# ──────────────────────────────────────────────────────────
# Helper functions
# ──────────────────────────────────────────────────────────
def clamp_tensor(x: torch.Tensor,
                 lower: torch.Tensor,
                 upper: torch.Tensor) -> torch.Tensor:
    """Clamp tensor values element‑wise between *lower* and *upper*."""
    return torch.max(torch.min(x, upper), lower)


def calculate_adversarial_loss(
    model: nn.Module,
    x_raw_single: torch.Tensor,
    y_single: torch.Tensor,
    eps: float = 1e-9
) -> torch.Tensor:
    """
    Black‑box cross‑entropy with ε‑floor (prevents CE == 0).

    Parameters
    ----------
    model  : nn.Module        – queried only in forward mode
    x_raw_single : [1, dim]   – raw pixels in [0,255]
    y_single     : [1]        – ground truth label
    eps          : float      – minimum CE to return
    """
    x_clip = clamp_tensor(x_raw_single, final_lower, final_upper)
    x_norm = x_clip / final_upper                       # scale to [0,1]

    if CNN_MODE:
        x_norm = x_norm.view(1, 1, *img_shape)          # keep CNN option

    # forward pass – still black‑box
    logits = model(x_norm)

    # CE for the single true class (no reduction over batch)
    log_probs = F.log_softmax(logits, dim=1)            # safe log
    ce = -log_probs[0, y_single]

    # floor to keep CE ≥ ε  (prevents fitness from flattening at 0)
    ce = torch.clamp(ce, min=eps)

    return ce



def save_uint8(img_arr: np.ndarray, path: str) -> None:
    """Round float32 array in [0,255] → uint8 and save with OpenCV."""
    img_u8 = np.clip(np.round(img_arr), final_lower_bound, final_upper_bound).astype(np.uint8)
    cv2.imwrite(path, img_u8)


# ──────────────────────────────────────────────────────────
# Load the trained model
# ──────────────────────────────────────────────────────────
MODEL_PATH = os.path.join("Models and Data splits", "MainModel_MLP1L.pt")

try:
    model = torch.jit.load(MODEL_PATH, map_location=device)
    model.to(device).eval()
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model from {MODEL_PATH}: {e}")
    model = None        # Guard in main()

# ──────────────────────────────────────────────────────────
# Cuckoo Search (single sample)
# ──────────────────────────────────────────────────────────
class CuckooSearchSingleSample:
    """
    Cuckoo‑Search meta‑heuristic to craft an adversarial perturbation
    for **one** image sample.
    """

    # --- constructor -------------------------------------------------
    def __init__(self,
                 n_nests: int,
                 n_iterations: int,
                 dim: int,
                 final_lower: torch.Tensor,
                 final_upper: torch.Tensor,
                 raw_lower: torch.Tensor,
                 raw_upper: torch.Tensor,
                 model: nn.Module,
                 X_sample: torch.Tensor,
                 y_sample: torch.Tensor,
                 p_a: float = 0.5,
                 lambda_param: float = 1e9,
                 step_size: float = 2.0,        
                 step_decay: float = 0.98):
        # Public params
        self.n_nests        = n_nests
        self.n_iterations   = n_iterations
        self.dim            = dim
        self.final_lower    = final_lower
        self.final_upper    = final_upper
        self.raw_lower      = raw_lower
        self.raw_upper      = raw_upper
        self.model          = model
        self.X              = X_sample.unsqueeze(0)   # [1, dim]
        self.y              = y_sample                # [1]
        self.p_a            = p_a
        self.lambda_param   = lambda_param
        self.step_size      = step_size               
        self.step_decay     = step_decay

        # --- population initialisation ------------------------------
        self.nests = torch.zeros((n_nests, dim), dtype=torch.float32, device=device)

        self.fitness = torch.empty(n_nests, dtype=torch.float32, device=device)
        for i in range(n_nests):
            self.fitness[i] = self._fitness(self.nests[i])

        best_idx = torch.argmin(self.fitness)
        self.best_nest    = self.nests[best_idx].clone()
        self.best_fitness = self.fitness[best_idx].item()

    # --- internals ---------------------------------------------------
    
    def _cauchy_flight(self) -> torch.Tensor:
        loc = torch.tensor(0.0, device=device)
        scale = torch.tensor(self.step_size, device=device)
        dist = torch.distributions.Cauchy(loc, scale)
        return dist.sample((self.dim,)) 

    def _new_solution(self, curr_perturb: torch.Tensor) -> torch.Tensor:
        flight = self._cauchy_flight()                              # [dim]
        cand   = clamp_tensor(curr_perturb + flight,
                              self.raw_lower, self.raw_upper)
        return cand

    def _fitness(self, perturb: torch.Tensor) -> float:
        """Fitness = ‖δ‖₂ − λ · CrossEntropy (untargeted)."""
        p_clip        = clamp_tensor(perturb, self.raw_lower, self.raw_upper)
        perturbed_raw = self.X + p_clip.unsqueeze(0)               # [1, dim]
        loss          = calculate_adversarial_loss(self.model,
                                                   perturbed_raw,
                                                   self.y).item()
        magnitude     = torch.norm(p_clip).item()
        return magnitude - self.lambda_param * loss

    # --- public API --------------------------------------------------
    def optimize(self) -> None:
        """Run Cuckoo‑Search for *n_iterations*."""
        for it in range(1, self.n_iterations + 1):
            # 1) generate new solutions
            for i in range(self.n_nests):
                cand      = self._new_solution(self.nests[i])
                cand_fit  = self._fitness(cand)

                if cand_fit < self.fitness[i]:
                    self.nests[i]  = cand
                    self.fitness[i]= cand_fit
                    if cand_fit < self.best_fitness:
                        self.best_fitness = cand_fit
                        self.best_nest    = cand.clone()

            # 2) abandon a fraction p_a of worst nests
            k = int(self.n_nests * self.p_a)
            if k > 0:
                worst_idx = torch.argsort(self.fitness)[-k:]
                for idx in worst_idx:
                    new_rand = torch.empty((self.dim,), dtype=torch.float32, device=device)\
                                  .uniform_(float(self.raw_lower),
                                            float(self.raw_upper))  
                    self.nests[idx]  = new_rand
                    self.fitness[idx]= self._fitness(new_rand)

            if it in {1, self.n_iterations} or it % 10 == 0:
                print(f"    iter {it}/{self.n_iterations} — best fitness {self.best_fitness:.5f}")

            # 3) step‑size schedule (optional)
            self.step_size *= self.step_decay                       

    def get_best_solution(self) -> torch.Tensor:
        """Return δ*  (shape [dim])."""
        return self.best_nest

# ──────────────────────────────────────────────────────────
# Main driver – sample‑by‑sample attack
# ──────────────────────────────────────────────────────────
def main() -> None:
    if model is None:
        print("Model not loaded; aborting.")
        return

    tic = time.time()
    print("Starting sample‑by‑sample adversarial generation …")

    # ---- load data --------------------------------------------------
    data_path = os.path.join("Models and Data splits", "MLP1L_500_samples_train.pkl")
    if not os.path.exists(data_path):
        print(f"Data file not found: {data_path}")
        return

    X_full_np, y_full_np = joblib.load(data_path)
    X_full = torch.tensor(X_full_np, dtype=torch.float32)
    y_full = torch.tensor(y_full_np, dtype=torch.long)
    print(f"Loaded X: {X_full.shape}, y: {y_full.shape}")

    # ---- output dirs -----------------------------------------------
    base_dir = "Generated Data/ZC-CSA_images"
    orig_dir = os.path.join(base_dir, "Original")
    adv_dir  = os.path.join(base_dir, "Adversarial")
    pert_dir = os.path.join(base_dir, "Perturbations")
    for d in (orig_dir, adv_dir, pert_dir):
        os.makedirs(d, exist_ok=True)

    # ---- hyper‑parameters ------------------------------------------
    n_nests       = 58
    n_iterations  = 100
    p_a           = 0.5
    lambda_param  = 1e10
    step_size     = 1.0      
    step_decay    = 0.98      # optional (1.0 disables decay)

    all_results = []
    processed   = 0
    adv_success = 0

    # ---- loop over digits ------------------------------------------
    for digit in range(10):     # process all digits now
        idxs = (y_full == digit).nonzero(as_tuple=True)[0]
        if idxs.numel() == 0:
            print(f"\nNo samples for digit {digit}; skipping.")
            continue

        print(f"\n── Processing digit {digit} ({idxs.numel()} samples) ──")
        X_class = X_full[idxs].to(device)
        y_class = y_full[idxs].to(device)

        for i in range(X_class.size(0)): 
            X_sample = X_class[i]                      # [dim]
            y_sample = y_class[i].unsqueeze(0)         # [1]
            gidx     = int(idxs[i])

            print(f"  sample {i+1}/{X_class.size(0)} (global {gidx})")

            # -- save original --------------------------------------
            save_uint8(X_sample.cpu().numpy().reshape(img_shape),
                       os.path.join(orig_dir, f"{gidx}_lbl{digit}.png"))

            # -- Cuckoo‑Search optimiser ---------------------------
            css = CuckooSearchSingleSample(
                n_nests      = n_nests,
                n_iterations = n_iterations,
                dim          = dim,
                final_lower  = final_lower,
                final_upper  = final_upper,
                raw_lower    = raw_lower,
                raw_upper    = raw_upper,
                model        = model,
                X_sample     = X_sample,
                y_sample     = y_sample,
                p_a          = p_a,
                lambda_param = lambda_param,
                step_size    = step_size,     
                step_decay   = step_decay,
            )
            css.optimize()
            delta_best = css.get_best_solution()

            # -- apply δ* ------------------------------------------
            adv_raw  = clamp_tensor(X_sample + delta_best, final_lower, final_upper)
            adv_norm = adv_raw.unsqueeze(0) / final_upper
            if CNN_MODE:
                adv_norm = adv_norm.view(1, 1, *img_shape)          

            with torch.no_grad():
                logits = model(adv_norm)
                pred   = torch.argmax(logits, dim=1)

            norm_l2  = torch.norm(delta_best).item()
            success  = (pred != y_sample).item()

            # -- save files ----------------------------------------
            delta_np = delta_best.cpu().numpy().reshape(img_shape)
            np.save(os.path.join(pert_dir, f"{gidx}_delta.npy"), delta_np)

            vis = ((delta_np - raw_lower_bound) / (raw_upper_bound - raw_lower_bound) * final_upper_bound)

            save_uint8(vis, os.path.join(pert_dir, f"{gidx}_delta_vis.png"))

            if success:
                adv_success += 1
                save_uint8(adv_raw.cpu().numpy().reshape(img_shape),
                           os.path.join(adv_dir,
                                        f"{gidx}_t{digit}_p{int(pred)}_m{norm_l2:.2f}.png"))

            # -- record row ----------------------------------------
            all_results.append(dict(
                digit=digit,
                index=gidx,
                true=digit,
                pred=int(pred),
                misclassified=success,
                perturb_norm_l2=norm_l2,
            ))

            processed += 1

    # ---- export CSV summary ----------------------------------------
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(base_dir, "zc_csa_500x50_results.csv")
    df.to_csv(csv_path, index=False)

    # ---- per‑digit statistics --------------------------------------
    stats = (df.groupby(["digit", "misclassified"])
               .size()
               .unstack(fill_value=0)
               .rename(columns={False: "correct", True: "misclassified"}))
    
    # Ensure both columns always exist
    if "misclassified" not in stats.columns:
        stats["misclassified"] = 0
    if "correct" not in stats.columns:
        stats["correct"] = 0
    
    stats["total"]      = stats["correct"] + stats["misclassified"]
    stats["correct_%"]  = (stats["correct"]       / stats["total"] * 100).round(2)
    stats["miscls_%"]   = (stats["misclassified"] / stats["total"] * 100).round(2)


    stats_csv = os.path.join(base_dir, "zc_csa_stats_per_digit.csv")
    stats.to_csv(stats_csv)

    toc = time.time()

    # ---- pretty print ---------------------------------------------
    print("\n===== SUMMARY =====")
    print(f"Processed samples        : {processed}")
    print(f"Successful attacks       : {adv_success}")
    print(f"Detailed results  CSV    : {csv_path}")
    print(f"Per‑digit stats   CSV    : {stats_csv}")
    print(f"Total time               : {toc - tic:.2f} s\n")

    print(stats.to_string())


if __name__ == "__main__":
    main()


Model loaded successfully from Models and Data splits\MainModel_MLP1L.pt
Starting sample‑by‑sample adversarial generation …
Loaded X: torch.Size([500, 784]), y: torch.Size([500])

── Processing digit 0 (50 samples) ──
  sample 1/50 (global 0)
    iter 1/100 — best fitness -10.00000
    iter 10/100 — best fitness -10.00000
    iter 20/100 — best fitness -10.00000
    iter 30/100 — best fitness -10.00000
    iter 40/100 — best fitness -10.00000
    iter 50/100 — best fitness -10.00000
    iter 60/100 — best fitness -10.00000
    iter 70/100 — best fitness -10.00000
    iter 80/100 — best fitness -10.00000
    iter 90/100 — best fitness -10.00000
    iter 100/100 — best fitness -10.00000
  sample 2/50 (global 1)
    iter 1/100 — best fitness -903.91118
    iter 10/100 — best fitness -1802411.84462
    iter 20/100 — best fitness -145407818.11292
    iter 30/100 — best fitness -1312163384.78008
    iter 40/100 — best fitness -14319328344.32617
    iter 50/100 — best fitness -31501459140.511

#### C-CSA

In [2]:
import os
import time
import joblib
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# ──────────────────────────────────────────────────────────
# Numeric bounds & device
# ──────────────────────────────────────────────────────────
raw_upper_bound  = 50.0
raw_lower_bound  = -70.0
final_lower_bound = 0.0
final_upper_bound = 255.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_lower  = torch.tensor(raw_lower_bound,  dtype=torch.float32, device=device)
raw_upper  = torch.tensor(raw_upper_bound,  dtype=torch.float32, device=device)
final_lower = torch.tensor(final_lower_bound, dtype=torch.float32, device=device)
final_upper = torch.tensor(final_upper_bound, dtype=torch.float32, device=device)

img_shape = (28, 28)
dim = img_shape[0] * img_shape[1]

# ──────────────────────────────────────────────────────────
# CNN switch
# ──────────────────────────────────────────────────────────
# Flip this to True when you load a convolutional network that
# expects input shaped [N, C, H, W] rather than flattened.
CNN_MODE = False           # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# ──────────────────────────────────────────────────────────
# Helper functions
# ──────────────────────────────────────────────────────────
def clamp_tensor(x: torch.Tensor,
                 lower: torch.Tensor,
                 upper: torch.Tensor) -> torch.Tensor:
    """Clamp tensor values element‑wise between *lower* and *upper*."""
    return torch.max(torch.min(x, upper), lower)


def calculate_adversarial_loss(
    model: nn.Module,
    x_raw_single: torch.Tensor,
    y_single: torch.Tensor,
    eps: float = 1e-9
) -> torch.Tensor:
    """
    Black‑box cross‑entropy with ε‑floor (prevents CE == 0).

    Parameters
    ----------
    model  : nn.Module        – queried only in forward mode
    x_raw_single : [1, dim]   – raw pixels in [0,255]
    y_single     : [1]        – ground truth label
    eps          : float      – minimum CE to return
    """
    x_clip = clamp_tensor(x_raw_single, final_lower, final_upper)
    x_norm = x_clip / final_upper                       # scale to [0,1]

    if CNN_MODE:
        x_norm = x_norm.view(1, 1, *img_shape)          # keep CNN option

    # forward pass – still black‑box
    logits = model(x_norm)

    # CE for the single true class (no reduction over batch)
    log_probs = F.log_softmax(logits, dim=1)            # safe log
    ce = -log_probs[0, y_single]

    # floor to keep CE ≥ ε  (prevents fitness from flattening at 0)
    ce = torch.clamp(ce, min=eps)

    return ce



def save_uint8(img_arr: np.ndarray, path: str) -> None:
    """Round float32 array in [0,255] → uint8 and save with OpenCV."""
    img_u8 = np.clip(np.round(img_arr), final_lower_bound, final_upper_bound).astype(np.uint8)
    cv2.imwrite(path, img_u8)


# ──────────────────────────────────────────────────────────
# Load the trained model
# ──────────────────────────────────────────────────────────
MODEL_PATH = os.path.join("Models and Data splits", "MainModel_MLP1L.pt")

try:
    model = torch.jit.load(MODEL_PATH, map_location=device)
    model.to(device).eval()
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model from {MODEL_PATH}: {e}")
    model = None        # Guard in main()

# ──────────────────────────────────────────────────────────
# Cuckoo Search (single sample)
# ──────────────────────────────────────────────────────────
class CuckooSearchSingleSample:
    """
    Cuckoo‑Search meta‑heuristic to craft an adversarial perturbation
    for **one** image sample.
    """

    # --- constructor -------------------------------------------------
    def __init__(self,
                 n_nests: int,
                 n_iterations: int,
                 dim: int,
                 final_lower: torch.Tensor,
                 final_upper: torch.Tensor,
                 raw_lower: torch.Tensor,
                 raw_upper: torch.Tensor,
                 model: nn.Module,
                 X_sample: torch.Tensor,
                 y_sample: torch.Tensor,
                 p_a: float = 0.5,
                 lambda_param: float = 1e9,
                 step_size: float = 2.0,        
                 step_decay: float = 0.98):
        # Public params
        self.n_nests        = n_nests
        self.n_iterations   = n_iterations
        self.dim            = dim
        self.final_lower    = final_lower
        self.final_upper    = final_upper
        self.raw_lower      = raw_lower
        self.raw_upper      = raw_upper
        self.model          = model
        self.X              = X_sample.unsqueeze(0)   # [1, dim]
        self.y              = y_sample                # [1]
        self.p_a            = p_a
        self.lambda_param   = lambda_param
        self.step_size      = step_size               
        self.step_decay     = step_decay

        # --- population initialisation ------------------------------
        self.nests = torch.empty((n_nests, dim), dtype=torch.float32, device=device) \
                 .uniform_(float(self.raw_lower), float(self.raw_upper))

        self.fitness = torch.empty(n_nests, dtype=torch.float32, device=device)
        for i in range(n_nests):
            self.fitness[i] = self._fitness(self.nests[i])

        best_idx = torch.argmin(self.fitness)
        self.best_nest    = self.nests[best_idx].clone()
        self.best_fitness = self.fitness[best_idx].item()

    # --- internals ---------------------------------------------------
    
    def _cauchy_flight(self) -> torch.Tensor:
        loc = torch.tensor(0.0, device=device)
        scale = torch.tensor(self.step_size, device=device)
        dist = torch.distributions.Cauchy(loc, scale)
        return dist.sample((self.dim,)) 

    def _new_solution(self, curr_perturb: torch.Tensor) -> torch.Tensor:
        flight = self._cauchy_flight()                              # [dim]
        cand   = clamp_tensor(curr_perturb + flight,
                              self.raw_lower, self.raw_upper)
        return cand

    def _fitness(self, perturb: torch.Tensor) -> float:
        """Fitness = ‖δ‖₂ − λ · CrossEntropy (untargeted)."""
        p_clip        = clamp_tensor(perturb, self.raw_lower, self.raw_upper)
        perturbed_raw = self.X + p_clip.unsqueeze(0)               # [1, dim]
        loss          = calculate_adversarial_loss(self.model,
                                                   perturbed_raw,
                                                   self.y).item()
        magnitude     = torch.norm(p_clip).item()
        return magnitude - self.lambda_param * loss

    # --- public API --------------------------------------------------
    def optimize(self) -> None:
        """Run Cuckoo‑Search for *n_iterations*."""
        for it in range(1, self.n_iterations + 1):
            # 1) generate new solutions
            for i in range(self.n_nests):
                cand      = self._new_solution(self.nests[i])
                cand_fit  = self._fitness(cand)

                if cand_fit < self.fitness[i]:
                    self.nests[i]  = cand
                    self.fitness[i]= cand_fit
                    if cand_fit < self.best_fitness:
                        self.best_fitness = cand_fit
                        self.best_nest    = cand.clone()

            # 2) abandon a fraction p_a of worst nests
            k = int(self.n_nests * self.p_a)
            if k > 0:
                worst_idx = torch.argsort(self.fitness)[-k:]
                for idx in worst_idx:
                    new_rand = torch.empty((self.dim,), dtype=torch.float32, device=device)\
                                  .uniform_(float(self.raw_lower),
                                            float(self.raw_upper))  
                    self.nests[idx]  = new_rand
                    self.fitness[idx]= self._fitness(new_rand)

            if it in {1, self.n_iterations} or it % 10 == 0:
                print(f"    iter {it}/{self.n_iterations} — best fitness {self.best_fitness:.5f}")

            # 3) step‑size schedule (optional)
            self.step_size *= self.step_decay                       

    def get_best_solution(self) -> torch.Tensor:
        """Return δ*  (shape [dim])."""
        return self.best_nest

# ──────────────────────────────────────────────────────────
# Main driver – sample‑by‑sample attack
# ──────────────────────────────────────────────────────────
def main() -> None:
    if model is None:
        print("Model not loaded; aborting.")
        return

    tic = time.time()
    print("Starting sample‑by‑sample adversarial generation …")

    # ---- load data --------------------------------------------------
    data_path = os.path.join("Models and Data splits", "MLP1L_500_samples_train.pkl")
    if not os.path.exists(data_path):
        print(f"Data file not found: {data_path}")
        return

    X_full_np, y_full_np = joblib.load(data_path)
    X_full = torch.tensor(X_full_np, dtype=torch.float32)
    y_full = torch.tensor(y_full_np, dtype=torch.long)
    print(f"Loaded X: {X_full.shape}, y: {y_full.shape}")

    # ---- output dirs -----------------------------------------------
    base_dir = "Generated Data/C-CSA_images"
    orig_dir = os.path.join(base_dir, "Original")
    adv_dir  = os.path.join(base_dir, "Adversarial")
    pert_dir = os.path.join(base_dir, "Perturbations")
    for d in (orig_dir, adv_dir, pert_dir):
        os.makedirs(d, exist_ok=True)

    # ---- hyper‑parameters ------------------------------------------
    n_nests       = 58
    n_iterations  = 100
    p_a           = 0.5
    lambda_param  = 1e10
    step_size     = 1.0      
    step_decay    = 0.98      # optional (1.0 disables decay)

    all_results = []
    processed   = 0
    adv_success = 0

    # ---- loop over digits ------------------------------------------
    for digit in range(10):     # process all digits now
        idxs = (y_full == digit).nonzero(as_tuple=True)[0]
        if idxs.numel() == 0:
            print(f"\nNo samples for digit {digit}; skipping.")
            continue

        print(f"\n── Processing digit {digit} ({idxs.numel()} samples) ──")
        X_class = X_full[idxs].to(device)
        y_class = y_full[idxs].to(device)

        for i in range(X_class.size(0)): 
            X_sample = X_class[i]                      # [dim]
            y_sample = y_class[i].unsqueeze(0)         # [1]
            gidx     = int(idxs[i])

            print(f"  sample {i+1}/{X_class.size(0)} (global {gidx})")

            # -- save original --------------------------------------
            save_uint8(X_sample.cpu().numpy().reshape(img_shape),
                       os.path.join(orig_dir, f"{gidx}_lbl{digit}.png"))

            # -- Cuckoo‑Search optimiser ---------------------------
            css = CuckooSearchSingleSample(
                n_nests      = n_nests,
                n_iterations = n_iterations,
                dim          = dim,
                final_lower  = final_lower,
                final_upper  = final_upper,
                raw_lower    = raw_lower,
                raw_upper    = raw_upper,
                model        = model,
                X_sample     = X_sample,
                y_sample     = y_sample,
                p_a          = p_a,
                lambda_param = lambda_param,
                step_size    = step_size,     
                step_decay   = step_decay,
            )
            css.optimize()
            delta_best = css.get_best_solution()

            # -- apply δ* ------------------------------------------
            adv_raw  = clamp_tensor(X_sample + delta_best, final_lower, final_upper)
            adv_norm = adv_raw.unsqueeze(0) / final_upper
            if CNN_MODE:
                adv_norm = adv_norm.view(1, 1, *img_shape)          

            with torch.no_grad():
                logits = model(adv_norm)
                pred   = torch.argmax(logits, dim=1)

            norm_l2  = torch.norm(delta_best).item()
            success  = (pred != y_sample).item()

            # -- save files ----------------------------------------
            delta_np = delta_best.cpu().numpy().reshape(img_shape)
            np.save(os.path.join(pert_dir, f"{gidx}_delta.npy"), delta_np)

            vis = ((delta_np - raw_lower_bound) / (raw_upper_bound - raw_lower_bound) * final_upper_bound)

            save_uint8(vis, os.path.join(pert_dir, f"{gidx}_delta_vis.png"))

            if success:
                adv_success += 1
                save_uint8(adv_raw.cpu().numpy().reshape(img_shape),
                           os.path.join(adv_dir,
                                        f"{gidx}_t{digit}_p{int(pred)}_m{norm_l2:.2f}.png"))

            # -- record row ----------------------------------------
            all_results.append(dict(
                digit=digit,
                index=gidx,
                true=digit,
                pred=int(pred),
                misclassified=success,
                perturb_norm_l2=norm_l2,
            ))

            processed += 1

    # ---- export CSV summary ----------------------------------------
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(base_dir, "c_csa_500x50_results.csv")
    df.to_csv(csv_path, index=False)

    # ---- per‑digit statistics --------------------------------------
    stats = (df.groupby(["digit", "misclassified"])
               .size()
               .unstack(fill_value=0)
               .rename(columns={False: "correct", True: "misclassified"}))
    
    # Ensure both columns always exist
    if "misclassified" not in stats.columns:
        stats["misclassified"] = 0
    if "correct" not in stats.columns:
        stats["correct"] = 0
    
    stats["total"]      = stats["correct"] + stats["misclassified"]
    stats["correct_%"]  = (stats["correct"]       / stats["total"] * 100).round(2)
    stats["miscls_%"]   = (stats["misclassified"] / stats["total"] * 100).round(2)


    stats_csv = os.path.join(base_dir, "c_csa_stats_per_digit.csv")
    stats.to_csv(stats_csv)

    toc = time.time()

    # ---- pretty print ---------------------------------------------
    print("\n===== SUMMARY =====")
    print(f"Processed samples        : {processed}")
    print(f"Successful attacks       : {adv_success}")
    print(f"Detailed results  CSV    : {csv_path}")
    print(f"Per‑digit stats   CSV    : {stats_csv}")
    print(f"Total time               : {toc - tic:.2f} s\n")

    print(stats.to_string())


if __name__ == "__main__":
    main()


Model loaded successfully from Models and Data splits\MainModel_MLP1L.pt
Starting sample‑by‑sample adversarial generation …
Loaded X: torch.Size([500, 784]), y: torch.Size([500])

── Processing digit 0 (50 samples) ──
  sample 1/50 (global 0)
    iter 1/100 — best fitness 945.98315
    iter 10/100 — best fitness 945.98315
    iter 20/100 — best fitness 945.98315
    iter 30/100 — best fitness 945.98315
    iter 40/100 — best fitness 945.98315
    iter 50/100 — best fitness 945.90131
    iter 60/100 — best fitness 943.82111
    iter 70/100 — best fitness 943.56244
    iter 80/100 — best fitness 943.15387
    iter 90/100 — best fitness 939.54785
    iter 100/100 — best fitness 932.88245
  sample 2/50 (global 1)
    iter 1/100 — best fitness -183771.75000
    iter 10/100 — best fitness -5917102.82177
    iter 20/100 — best fitness -421301917.12657
    iter 30/100 — best fitness -6677506660.84222
    iter 40/100 — best fitness -15847215731.68091
    iter 50/100 — best fitness -37797094648.

#### Z-CSA

In [5]:
import os
import time
import joblib
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# ──────────────────────────────────────────────────────────
# Numeric bounds & device
# ──────────────────────────────────────────────────────────
raw_upper_bound  = 50.0
raw_lower_bound  = -70.0
final_lower_bound = 0.0
final_upper_bound = 255.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_lower  = torch.tensor(raw_lower_bound,  dtype=torch.float32, device=device)
raw_upper  = torch.tensor(raw_upper_bound,  dtype=torch.float32, device=device)
final_lower = torch.tensor(final_lower_bound, dtype=torch.float32, device=device)
final_upper = torch.tensor(final_upper_bound, dtype=torch.float32, device=device)

img_shape = (28, 28)
dim = img_shape[0] * img_shape[1]

# ──────────────────────────────────────────────────────────
# CNN switch
# ──────────────────────────────────────────────────────────
# Flip this to True when you load a convolutional network that
# expects input shaped [N, C, H, W] rather than flattened.
CNN_MODE = False           # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# ──────────────────────────────────────────────────────────
# Helper functions
# ──────────────────────────────────────────────────────────
def clamp_tensor(x: torch.Tensor,
                 lower: torch.Tensor,
                 upper: torch.Tensor) -> torch.Tensor:
    """Clamp tensor values element‑wise between *lower* and *upper*."""
    return torch.max(torch.min(x, upper), lower)


def calculate_adversarial_loss(
    model: nn.Module,
    x_raw_single: torch.Tensor,
    y_single: torch.Tensor,
    eps: float = 1e-9
) -> torch.Tensor:
    """
    Black‑box cross‑entropy with ε‑floor (prevents CE == 0).

    Parameters
    ----------
    model  : nn.Module        – queried only in forward mode
    x_raw_single : [1, dim]   – raw pixels in [0,255]
    y_single     : [1]        – ground truth label
    eps          : float      – minimum CE to return
    """
    x_clip = clamp_tensor(x_raw_single, final_lower, final_upper)
    x_norm = x_clip / final_upper                       # scale to [0,1]

    if CNN_MODE:
        x_norm = x_norm.view(1, 1, *img_shape)          # keep CNN option

    # forward pass – still black‑box
    logits = model(x_norm)

    # CE for the single true class (no reduction over batch)
    log_probs = F.log_softmax(logits, dim=1)            # safe log
    ce = -log_probs[0, y_single]

    # floor to keep CE ≥ ε  (prevents fitness from flattening at 0)
    ce = torch.clamp(ce, min=eps)

    return ce



def save_uint8(img_arr: np.ndarray, path: str) -> None:
    """Round float32 array in [0,255] → uint8 and save with OpenCV."""
    img_u8 = np.clip(np.round(img_arr), final_lower_bound, final_upper_bound).astype(np.uint8)
    cv2.imwrite(path, img_u8)


# ──────────────────────────────────────────────────────────
# Load the trained model
# ──────────────────────────────────────────────────────────
MODEL_PATH = os.path.join("Models and Data splits", "MainModel_MLP1L.pt")

try:
    model = torch.jit.load(MODEL_PATH, map_location=device)
    model.to(device).eval()
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model from {MODEL_PATH}: {e}")
    model = None        # Guard in main()

# ──────────────────────────────────────────────────────────
# Cuckoo Search (single sample)
# ──────────────────────────────────────────────────────────
class CuckooSearchSingleSample:
    """
    Cuckoo‑Search meta‑heuristic to craft an adversarial perturbation
    for **one** image sample.
    """

    # --- constructor -------------------------------------------------
    def __init__(self,
                 n_nests: int,
                 n_iterations: int,
                 dim: int,
                 final_lower: torch.Tensor,
                 final_upper: torch.Tensor,
                 raw_lower: torch.Tensor,
                 raw_upper: torch.Tensor,
                 model: nn.Module,
                 X_sample: torch.Tensor,
                 y_sample: torch.Tensor,
                 p_a: float = 0.5,
                 lambda_param: float = 1e9,
                 step_size: float = 2.0,        
                 step_decay: float = 0.98):
        # Public params
        self.n_nests        = n_nests
        self.n_iterations   = n_iterations
        self.dim            = dim
        self.final_lower    = final_lower
        self.final_upper    = final_upper
        self.raw_lower      = raw_lower
        self.raw_upper      = raw_upper
        self.model          = model
        self.X              = X_sample.unsqueeze(0)   # [1, dim]
        self.y              = y_sample                # [1]
        self.p_a            = p_a
        self.lambda_param   = lambda_param
        self.step_size      = step_size               
        self.step_decay     = step_decay

        # --- population initialisation ------------------------------
        self.nests = torch.zeros((n_nests, dim), dtype=torch.float32, device=device)

        self.fitness = torch.empty(n_nests, dtype=torch.float32, device=device)
        for i in range(n_nests):
            self.fitness[i] = self._fitness(self.nests[i])

        best_idx = torch.argmin(self.fitness)
        self.best_nest    = self.nests[best_idx].clone()
        self.best_fitness = self.fitness[best_idx].item()

    # --- internals ---------------------------------------------------
    
    def _levy_flight(self) -> torch.Tensor:
        # Lévy flight (Mantegna’s algorithm, β = 1.5)
        import math
        beta  = 1.5
        sigma = (math.gamma(1 + beta) * math.sin(np.pi * beta / 2) /
                 (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u     = torch.randn(self.dim, device=device) * (sigma * self.step_size)
        v     = torch.randn(self.dim, device=device)
        step  = u / torch.pow(torch.abs(v), 1 / beta)
        return step


    def _new_solution(self, curr_perturb: torch.Tensor) -> torch.Tensor:
        flight = self._levy_flight()                              # [dim]
        cand   = clamp_tensor(curr_perturb + flight,
                              self.raw_lower, self.raw_upper)
        return cand

    def _fitness(self, perturb: torch.Tensor) -> float:
        """Fitness = ‖δ‖₂ − λ · CrossEntropy (untargeted)."""
        p_clip        = clamp_tensor(perturb, self.raw_lower, self.raw_upper)
        perturbed_raw = self.X + p_clip.unsqueeze(0)               # [1, dim]
        loss          = calculate_adversarial_loss(self.model,
                                                   perturbed_raw,
                                                   self.y).item()
        magnitude     = torch.norm(p_clip).item()
        return magnitude - self.lambda_param * loss

    # --- public API --------------------------------------------------
    def optimize(self) -> None:
        """Run Cuckoo‑Search for *n_iterations*."""
        for it in range(1, self.n_iterations + 1):
            # 1) generate new solutions
            for i in range(self.n_nests):
                cand      = self._new_solution(self.nests[i])
                cand_fit  = self._fitness(cand)

                if cand_fit < self.fitness[i]:
                    self.nests[i]  = cand
                    self.fitness[i]= cand_fit
                    if cand_fit < self.best_fitness:
                        self.best_fitness = cand_fit
                        self.best_nest    = cand.clone()

            # 2) abandon a fraction p_a of worst nests
            k = int(self.n_nests * self.p_a)
            if k > 0:
                worst_idx = torch.argsort(self.fitness)[-k:]
                for idx in worst_idx:
                    new_rand = torch.empty((self.dim,), dtype=torch.float32, device=device)\
                                  .uniform_(float(self.raw_lower),
                                            float(self.raw_upper))  
                    self.nests[idx]  = new_rand
                    self.fitness[idx]= self._fitness(new_rand)

            if it in {1, self.n_iterations} or it % 10 == 0:
                print(f"    iter {it}/{self.n_iterations} — best fitness {self.best_fitness:.5f}")

            # 3) step‑size schedule (optional)
            self.step_size *= self.step_decay                       

    def get_best_solution(self) -> torch.Tensor:
        """Return δ*  (shape [dim])."""
        return self.best_nest

# ──────────────────────────────────────────────────────────
# Main driver – sample‑by‑sample attack
# ──────────────────────────────────────────────────────────
def main() -> None:
    if model is None:
        print("Model not loaded; aborting.")
        return

    tic = time.time()
    print("Starting sample‑by‑sample adversarial generation …")

    # ---- load data --------------------------------------------------
    data_path = os.path.join("Models and Data splits", "MLP1L_500_samples_train.pkl")
    if not os.path.exists(data_path):
        print(f"Data file not found: {data_path}")
        return

    X_full_np, y_full_np = joblib.load(data_path)
    X_full = torch.tensor(X_full_np, dtype=torch.float32)
    y_full = torch.tensor(y_full_np, dtype=torch.long)
    print(f"Loaded X: {X_full.shape}, y: {y_full.shape}")

    # ---- output dirs -----------------------------------------------
    base_dir = "Generated Data/Z-CSA_images"
    orig_dir = os.path.join(base_dir, "Original")
    adv_dir  = os.path.join(base_dir, "Adversarial")
    pert_dir = os.path.join(base_dir, "Perturbations")
    for d in (orig_dir, adv_dir, pert_dir):
        os.makedirs(d, exist_ok=True)

    # ---- hyper‑parameters ------------------------------------------
    n_nests       = 58
    n_iterations  = 100
    p_a           = 0.5
    lambda_param  = 1e10
    step_size     = 1.0      
    step_decay    = 0.98      # optional (1.0 disables decay)

    all_results = []
    processed   = 0
    adv_success = 0

    # ---- loop over digits ------------------------------------------
    for digit in range(10):     # process all digits now
        idxs = (y_full == digit).nonzero(as_tuple=True)[0]
        if idxs.numel() == 0:
            print(f"\nNo samples for digit {digit}; skipping.")
            continue

        print(f"\n── Processing digit {digit} ({idxs.numel()} samples) ──")
        X_class = X_full[idxs].to(device)
        y_class = y_full[idxs].to(device)

        for i in range(X_class.size(0)): 
            X_sample = X_class[i]                      # [dim]
            y_sample = y_class[i].unsqueeze(0)         # [1]
            gidx     = int(idxs[i])

            print(f"  sample {i+1}/{X_class.size(0)} (global {gidx})")

            # -- save original --------------------------------------
            save_uint8(X_sample.cpu().numpy().reshape(img_shape),
                       os.path.join(orig_dir, f"{gidx}_lbl{digit}.png"))

            # -- Cuckoo‑Search optimiser ---------------------------
            css = CuckooSearchSingleSample(
                n_nests      = n_nests,
                n_iterations = n_iterations,
                dim          = dim,
                final_lower  = final_lower,
                final_upper  = final_upper,
                raw_lower    = raw_lower,
                raw_upper    = raw_upper,
                model        = model,
                X_sample     = X_sample,
                y_sample     = y_sample,
                p_a          = p_a,
                lambda_param = lambda_param,
                step_size    = step_size,     
                step_decay   = step_decay,
            )
            css.optimize()
            delta_best = css.get_best_solution()

            # -- apply δ* ------------------------------------------
            adv_raw  = clamp_tensor(X_sample + delta_best, final_lower, final_upper)
            adv_norm = adv_raw.unsqueeze(0) / final_upper
            if CNN_MODE:
                adv_norm = adv_norm.view(1, 1, *img_shape)          

            with torch.no_grad():
                logits = model(adv_norm)
                pred   = torch.argmax(logits, dim=1)

            norm_l2  = torch.norm(delta_best).item()
            success  = (pred != y_sample).item()

            # -- save files ----------------------------------------
            delta_np = delta_best.cpu().numpy().reshape(img_shape)
            np.save(os.path.join(pert_dir, f"{gidx}_delta.npy"), delta_np)

            vis = ((delta_np - raw_lower_bound) / (raw_upper_bound - raw_lower_bound) * final_upper_bound)

            save_uint8(vis, os.path.join(pert_dir, f"{gidx}_delta_vis.png"))

            if success:
                adv_success += 1
                save_uint8(adv_raw.cpu().numpy().reshape(img_shape),
                           os.path.join(adv_dir,
                                        f"{gidx}_t{digit}_p{int(pred)}_m{norm_l2:.2f}.png"))

            # -- record row ----------------------------------------
            all_results.append(dict(
                digit=digit,
                index=gidx,
                true=digit,
                pred=int(pred),
                misclassified=success,
                perturb_norm_l2=norm_l2,
            ))

            processed += 1

    # ---- export CSV summary ----------------------------------------
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(base_dir, "z_csa_500x50_results.csv")
    df.to_csv(csv_path, index=False)

    # ---- per‑digit statistics --------------------------------------
    stats = (df.groupby(["digit", "misclassified"])
               .size()
               .unstack(fill_value=0)
               .rename(columns={False: "correct", True: "misclassified"}))
    
    # Ensure both columns always exist
    if "misclassified" not in stats.columns:
        stats["misclassified"] = 0
    if "correct" not in stats.columns:
        stats["correct"] = 0
    
    stats["total"]      = stats["correct"] + stats["misclassified"]
    stats["correct_%"]  = (stats["correct"]       / stats["total"] * 100).round(2)
    stats["miscls_%"]   = (stats["misclassified"] / stats["total"] * 100).round(2)


    stats_csv = os.path.join(base_dir, "z_csa_stats_per_digit.csv")
    stats.to_csv(stats_csv)

    toc = time.time()

    # ---- pretty print ---------------------------------------------
    print("\n===== SUMMARY =====")
    print(f"Processed samples        : {processed}")
    print(f"Successful attacks       : {adv_success}")
    print(f"Detailed results  CSV    : {csv_path}")
    print(f"Per‑digit stats   CSV    : {stats_csv}")
    print(f"Total time               : {toc - tic:.2f} s\n")

    print(stats.to_string())


if __name__ == "__main__":
    main()


Model loaded successfully from Models and Data splits\MainModel_MLP1L.pt
Starting sample‑by‑sample adversarial generation …
Loaded X: torch.Size([500, 784]), y: torch.Size([500])

── Processing digit 0 (50 samples) ──
  sample 1/50 (global 0)
    iter 1/100 — best fitness -10.00000
    iter 10/100 — best fitness -10.00000
    iter 20/100 — best fitness -10.00000
    iter 30/100 — best fitness -10.00000
    iter 40/100 — best fitness -10.00000
    iter 50/100 — best fitness -10.00000
    iter 60/100 — best fitness -10.00000
    iter 70/100 — best fitness -10.00000
    iter 80/100 — best fitness -10.00000
    iter 90/100 — best fitness -10.00000
    iter 100/100 — best fitness -10.00000
  sample 2/50 (global 1)
    iter 1/100 — best fitness -10.00000
    iter 10/100 — best fitness -244544.00086
    iter 20/100 — best fitness -7988761.41307
    iter 30/100 — best fitness -18326101.66924
    iter 40/100 — best fitness -70223586.36649
    iter 50/100 — best fitness -275821267.52458
    iter

#### CSA

In [6]:
import os
import time
import joblib
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# ──────────────────────────────────────────────────────────
# Numeric bounds & device
# ──────────────────────────────────────────────────────────
raw_upper_bound  = 50.0
raw_lower_bound  = -70.0
final_lower_bound = 0.0
final_upper_bound = 255.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_lower  = torch.tensor(raw_lower_bound,  dtype=torch.float32, device=device)
raw_upper  = torch.tensor(raw_upper_bound,  dtype=torch.float32, device=device)
final_lower = torch.tensor(final_lower_bound, dtype=torch.float32, device=device)
final_upper = torch.tensor(final_upper_bound, dtype=torch.float32, device=device)

img_shape = (28, 28)
dim = img_shape[0] * img_shape[1]

# ──────────────────────────────────────────────────────────
# CNN switch
# ──────────────────────────────────────────────────────────
# Flip this to True when you load a convolutional network that
# expects input shaped [N, C, H, W] rather than flattened.
CNN_MODE = False           # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# ──────────────────────────────────────────────────────────
# Helper functions
# ──────────────────────────────────────────────────────────
def clamp_tensor(x: torch.Tensor,
                 lower: torch.Tensor,
                 upper: torch.Tensor) -> torch.Tensor:
    """Clamp tensor values element‑wise between *lower* and *upper*."""
    return torch.max(torch.min(x, upper), lower)


def calculate_adversarial_loss(
    model: nn.Module,
    x_raw_single: torch.Tensor,
    y_single: torch.Tensor,
    eps: float = 1e-9
) -> torch.Tensor:
    """
    Black‑box cross‑entropy with ε‑floor (prevents CE == 0).

    Parameters
    ----------
    model  : nn.Module        – queried only in forward mode
    x_raw_single : [1, dim]   – raw pixels in [0,255]
    y_single     : [1]        – ground truth label
    eps          : float      – minimum CE to return
    """
    x_clip = clamp_tensor(x_raw_single, final_lower, final_upper)
    x_norm = x_clip / final_upper                       # scale to [0,1]

    if CNN_MODE:
        x_norm = x_norm.view(1, 1, *img_shape)          # keep CNN option

    # forward pass – still black‑box
    logits = model(x_norm)

    # CE for the single true class (no reduction over batch)
    log_probs = F.log_softmax(logits, dim=1)            # safe log
    ce = -log_probs[0, y_single]

    # floor to keep CE ≥ ε  (prevents fitness from flattening at 0)
    ce = torch.clamp(ce, min=eps)

    return ce



def save_uint8(img_arr: np.ndarray, path: str) -> None:
    """Round float32 array in [0,255] → uint8 and save with OpenCV."""
    img_u8 = np.clip(np.round(img_arr), final_lower_bound, final_upper_bound).astype(np.uint8)
    cv2.imwrite(path, img_u8)


# ──────────────────────────────────────────────────────────
# Load the trained model
# ──────────────────────────────────────────────────────────
MODEL_PATH = os.path.join("Models and Data splits", "MainModel_MLP1L.pt")

try:
    model = torch.jit.load(MODEL_PATH, map_location=device)
    model.to(device).eval()
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model from {MODEL_PATH}: {e}")
    model = None        # Guard in main()

# ──────────────────────────────────────────────────────────
# Cuckoo Search (single sample)
# ──────────────────────────────────────────────────────────
class CuckooSearchSingleSample:
    """
    Cuckoo‑Search meta‑heuristic to craft an adversarial perturbation
    for **one** image sample.
    """

    # --- constructor -------------------------------------------------
    def __init__(self,
                 n_nests: int,
                 n_iterations: int,
                 dim: int,
                 final_lower: torch.Tensor,
                 final_upper: torch.Tensor,
                 raw_lower: torch.Tensor,
                 raw_upper: torch.Tensor,
                 model: nn.Module,
                 X_sample: torch.Tensor,
                 y_sample: torch.Tensor,
                 p_a: float = 0.5,
                 lambda_param: float = 1e9,
                 step_size: float = 2.0,        
                 step_decay: float = 0.98):
        # Public params
        self.n_nests        = n_nests
        self.n_iterations   = n_iterations
        self.dim            = dim
        self.final_lower    = final_lower
        self.final_upper    = final_upper
        self.raw_lower      = raw_lower
        self.raw_upper      = raw_upper
        self.model          = model
        self.X              = X_sample.unsqueeze(0)   # [1, dim]
        self.y              = y_sample                # [1]
        self.p_a            = p_a
        self.lambda_param   = lambda_param
        self.step_size      = step_size               
        self.step_decay     = step_decay

        # --- population initialisation ------------------------------
        self.nests = torch.empty((n_nests, dim), dtype=torch.float32, device=device) \
                 .uniform_(float(self.raw_lower), float(self.raw_upper))

        self.fitness = torch.empty(n_nests, dtype=torch.float32, device=device)
        for i in range(n_nests):
            self.fitness[i] = self._fitness(self.nests[i])

        best_idx = torch.argmin(self.fitness)
        self.best_nest    = self.nests[best_idx].clone()
        self.best_fitness = self.fitness[best_idx].item()

    # --- internals ---------------------------------------------------
    
    def _levy_flight(self) -> torch.Tensor:
        # Lévy flight (Mantegna’s algorithm, β = 1.5)
        import math
        beta  = 1.5
        sigma = (math.gamma(1 + beta) * math.sin(np.pi * beta / 2) /
                 (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u     = torch.randn(self.dim, device=device) * (sigma * self.step_size)
        v     = torch.randn(self.dim, device=device)
        step  = u / torch.pow(torch.abs(v), 1 / beta)
        return step


    def _new_solution(self, curr_perturb: torch.Tensor) -> torch.Tensor:
        flight = self._levy_flight()                              # [dim]
        cand   = clamp_tensor(curr_perturb + flight,
                              self.raw_lower, self.raw_upper)
        return cand

    def _fitness(self, perturb: torch.Tensor) -> float:
        """Fitness = ‖δ‖₂ − λ · CrossEntropy (untargeted)."""
        p_clip        = clamp_tensor(perturb, self.raw_lower, self.raw_upper)
        perturbed_raw = self.X + p_clip.unsqueeze(0)               # [1, dim]
        loss          = calculate_adversarial_loss(self.model,
                                                   perturbed_raw,
                                                   self.y).item()
        magnitude     = torch.norm(p_clip).item()
        return magnitude - self.lambda_param * loss

    # --- public API --------------------------------------------------
    def optimize(self) -> None:
        """Run Cuckoo‑Search for *n_iterations*."""
        for it in range(1, self.n_iterations + 1):
            # 1) generate new solutions
            for i in range(self.n_nests):
                cand      = self._new_solution(self.nests[i])
                cand_fit  = self._fitness(cand)

                if cand_fit < self.fitness[i]:
                    self.nests[i]  = cand
                    self.fitness[i]= cand_fit
                    if cand_fit < self.best_fitness:
                        self.best_fitness = cand_fit
                        self.best_nest    = cand.clone()

            # 2) abandon a fraction p_a of worst nests
            k = int(self.n_nests * self.p_a)
            if k > 0:
                worst_idx = torch.argsort(self.fitness)[-k:]
                for idx in worst_idx:
                    new_rand = torch.empty((self.dim,), dtype=torch.float32, device=device)\
                                  .uniform_(float(self.raw_lower),
                                            float(self.raw_upper))  
                    self.nests[idx]  = new_rand
                    self.fitness[idx]= self._fitness(new_rand)

            if it in {1, self.n_iterations} or it % 10 == 0:
                print(f"    iter {it}/{self.n_iterations} — best fitness {self.best_fitness:.5f}")

            # 3) step‑size schedule (optional)
            self.step_size *= self.step_decay                       

    def get_best_solution(self) -> torch.Tensor:
        """Return δ*  (shape [dim])."""
        return self.best_nest

# ──────────────────────────────────────────────────────────
# Main driver – sample‑by‑sample attack
# ──────────────────────────────────────────────────────────
def main() -> None:
    if model is None:
        print("Model not loaded; aborting.")
        return

    tic = time.time()
    print("Starting sample‑by‑sample adversarial generation …")

    # ---- load data --------------------------------------------------
    data_path = os.path.join("Models and Data splits", "MLP1L_500_samples_train.pkl")
    if not os.path.exists(data_path):
        print(f"Data file not found: {data_path}")
        return

    X_full_np, y_full_np = joblib.load(data_path)
    X_full = torch.tensor(X_full_np, dtype=torch.float32)
    y_full = torch.tensor(y_full_np, dtype=torch.long)
    print(f"Loaded X: {X_full.shape}, y: {y_full.shape}")

    # ---- output dirs -----------------------------------------------
    base_dir = "Generated Data/CSA_images"
    orig_dir = os.path.join(base_dir, "Original")
    adv_dir  = os.path.join(base_dir, "Adversarial")
    pert_dir = os.path.join(base_dir, "Perturbations")
    for d in (orig_dir, adv_dir, pert_dir):
        os.makedirs(d, exist_ok=True)

    # ---- hyper‑parameters ------------------------------------------
    n_nests       = 58
    n_iterations  = 100
    p_a           = 0.5
    lambda_param  = 1e10
    step_size     = 1.0      
    step_decay    = 0.98      # optional (1.0 disables decay)

    all_results = []
    processed   = 0
    adv_success = 0

    # ---- loop over digits ------------------------------------------
    for digit in range(10):     # process all digits now
        idxs = (y_full == digit).nonzero(as_tuple=True)[0]
        if idxs.numel() == 0:
            print(f"\nNo samples for digit {digit}; skipping.")
            continue

        print(f"\n── Processing digit {digit} ({idxs.numel()} samples) ──")
        X_class = X_full[idxs].to(device)
        y_class = y_full[idxs].to(device)

        for i in range(X_class.size(0)): 
            X_sample = X_class[i]                      # [dim]
            y_sample = y_class[i].unsqueeze(0)         # [1]
            gidx     = int(idxs[i])

            print(f"  sample {i+1}/{X_class.size(0)} (global {gidx})")

            # -- save original --------------------------------------
            save_uint8(X_sample.cpu().numpy().reshape(img_shape),
                       os.path.join(orig_dir, f"{gidx}_lbl{digit}.png"))

            # -- Cuckoo‑Search optimiser ---------------------------
            css = CuckooSearchSingleSample(
                n_nests      = n_nests,
                n_iterations = n_iterations,
                dim          = dim,
                final_lower  = final_lower,
                final_upper  = final_upper,
                raw_lower    = raw_lower,
                raw_upper    = raw_upper,
                model        = model,
                X_sample     = X_sample,
                y_sample     = y_sample,
                p_a          = p_a,
                lambda_param = lambda_param,
                step_size    = step_size,     
                step_decay   = step_decay,
            )
            css.optimize()
            delta_best = css.get_best_solution()

            # -- apply δ* ------------------------------------------
            adv_raw  = clamp_tensor(X_sample + delta_best, final_lower, final_upper)
            adv_norm = adv_raw.unsqueeze(0) / final_upper
            if CNN_MODE:
                adv_norm = adv_norm.view(1, 1, *img_shape)          

            with torch.no_grad():
                logits = model(adv_norm)
                pred   = torch.argmax(logits, dim=1)

            norm_l2  = torch.norm(delta_best).item()
            success  = (pred != y_sample).item()

            # -- save files ----------------------------------------
            delta_np = delta_best.cpu().numpy().reshape(img_shape)
            np.save(os.path.join(pert_dir, f"{gidx}_delta.npy"), delta_np)

            vis = ((delta_np - raw_lower_bound) / (raw_upper_bound - raw_lower_bound) * final_upper_bound)

            save_uint8(vis, os.path.join(pert_dir, f"{gidx}_delta_vis.png"))

            if success:
                adv_success += 1
                save_uint8(adv_raw.cpu().numpy().reshape(img_shape),
                           os.path.join(adv_dir,
                                        f"{gidx}_t{digit}_p{int(pred)}_m{norm_l2:.2f}.png"))

            # -- record row ----------------------------------------
            all_results.append(dict(
                digit=digit,
                index=gidx,
                true=digit,
                pred=int(pred),
                misclassified=success,
                perturb_norm_l2=norm_l2,
            ))

            processed += 1

    # ---- export CSV summary ----------------------------------------
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(base_dir, "csa_500x50_results.csv")
    df.to_csv(csv_path, index=False)

    # ---- per‑digit statistics --------------------------------------
    stats = (df.groupby(["digit", "misclassified"])
               .size()
               .unstack(fill_value=0)
               .rename(columns={False: "correct", True: "misclassified"}))
    
    # Ensure both columns always exist
    if "misclassified" not in stats.columns:
        stats["misclassified"] = 0
    if "correct" not in stats.columns:
        stats["correct"] = 0
    
    stats["total"]      = stats["correct"] + stats["misclassified"]
    stats["correct_%"]  = (stats["correct"]       / stats["total"] * 100).round(2)
    stats["miscls_%"]   = (stats["misclassified"] / stats["total"] * 100).round(2)


    stats_csv = os.path.join(base_dir, "csa_stats_per_digit.csv")
    stats.to_csv(stats_csv)

    toc = time.time()

    # ---- pretty print ---------------------------------------------
    print("\n===== SUMMARY =====")
    print(f"Processed samples        : {processed}")
    print(f"Successful attacks       : {adv_success}")
    print(f"Detailed results  CSV    : {csv_path}")
    print(f"Per‑digit stats   CSV    : {stats_csv}")
    print(f"Total time               : {toc - tic:.2f} s\n")

    print(stats.to_string())


if __name__ == "__main__":
    main()


Model loaded successfully from Models and Data splits\MainModel_MLP1L.pt
Starting sample‑by‑sample adversarial generation …
Loaded X: torch.Size([500, 784]), y: torch.Size([500])

── Processing digit 0 (50 samples) ──
  sample 1/50 (global 0)
    iter 1/100 — best fitness 951.23340
    iter 10/100 — best fitness 948.35675
    iter 20/100 — best fitness 946.62201
    iter 30/100 — best fitness 940.98108
    iter 40/100 — best fitness 936.56677
    iter 50/100 — best fitness 930.63208
    iter 60/100 — best fitness 929.62927
    iter 70/100 — best fitness 924.09027
    iter 80/100 — best fitness 920.62000
    iter 90/100 — best fitness 920.06964
    iter 100/100 — best fitness 918.00397
  sample 2/50 (global 1)
    iter 1/100 — best fitness -75287.91406
    iter 10/100 — best fitness -739243.03796
    iter 20/100 — best fitness -43478906.24034
    iter 30/100 — best fitness -48952667.57620
    iter 40/100 — best fitness -86723294.04433
    iter 50/100 — best fitness -254646652.72703
    

## Perturbation on 500 Samples from test/unseen set

In [3]:
import os
import time
import joblib
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# ──────────────────────────────────────────────────────────
# Numeric bounds & device
# ──────────────────────────────────────────────────────────
raw_upper_bound  = 50.0
raw_lower_bound  = -70.0
final_lower_bound = 0.0
final_upper_bound = 255.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_lower  = torch.tensor(raw_lower_bound,  dtype=torch.float32, device=device)
raw_upper  = torch.tensor(raw_upper_bound,  dtype=torch.float32, device=device)
final_lower = torch.tensor(final_lower_bound, dtype=torch.float32, device=device)
final_upper = torch.tensor(final_upper_bound, dtype=torch.float32, device=device)

img_shape = (28, 28)
dim = img_shape[0] * img_shape[1]

# ──────────────────────────────────────────────────────────
# CNN switch
# ──────────────────────────────────────────────────────────
# Flip this to True when you load a convolutional network that
# expects input shaped [N, C, H, W] rather than flattened.
CNN_MODE = False           # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# ──────────────────────────────────────────────────────────
# Helper functions
# ──────────────────────────────────────────────────────────
def clamp_tensor(x: torch.Tensor,
                 lower: torch.Tensor,
                 upper: torch.Tensor) -> torch.Tensor:
    """Clamp tensor values element‑wise between *lower* and *upper*."""
    return torch.max(torch.min(x, upper), lower)


def calculate_adversarial_loss(
    model: nn.Module,
    x_raw_single: torch.Tensor,
    y_single: torch.Tensor,
    eps: float = 1e-9
) -> torch.Tensor:
    """
    Black‑box cross‑entropy with ε‑floor (prevents CE == 0).

    Parameters
    ----------
    model  : nn.Module        – queried only in forward mode
    x_raw_single : [1, dim]   – raw pixels in [0,255]
    y_single     : [1]        – ground truth label
    eps          : float      – minimum CE to return
    """
    x_clip = clamp_tensor(x_raw_single, final_lower, final_upper)
    x_norm = x_clip / final_upper                       # scale to [0,1]

    if CNN_MODE:
        x_norm = x_norm.view(1, 1, *img_shape)          # keep CNN option

    # forward pass – still black‑box
    logits = model(x_norm)

    # CE for the single true class (no reduction over batch)
    log_probs = F.log_softmax(logits, dim=1)            # safe log
    ce = -log_probs[0, y_single]

    # floor to keep CE ≥ ε  (prevents fitness from flattening at 0)
    ce = torch.clamp(ce, min=eps)

    return ce



def save_uint8(img_arr: np.ndarray, path: str) -> None:
    """Round float32 array in [0,255] → uint8 and save with OpenCV."""
    img_u8 = np.clip(np.round(img_arr), final_lower_bound, final_upper_bound).astype(np.uint8)
    cv2.imwrite(path, img_u8)


# ──────────────────────────────────────────────────────────
# Load the trained model
# ──────────────────────────────────────────────────────────
MODEL_PATH = os.path.join("Models and Data splits", "MainModel_MLP1L.pt")

try:
    model = torch.jit.load(MODEL_PATH, map_location=device)
    model.to(device).eval()
    print(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    print(f"Error loading model from {MODEL_PATH}: {e}")
    model = None        # Guard in main()

# ──────────────────────────────────────────────────────────
# Cuckoo Search (single sample)
# ──────────────────────────────────────────────────────────
class CuckooSearchSingleSample:
    """
    Cuckoo‑Search meta‑heuristic to craft an adversarial perturbation
    for **one** image sample.
    """

    # --- constructor -------------------------------------------------
    def __init__(self,
                 n_nests: int,
                 n_iterations: int,
                 dim: int,
                 final_lower: torch.Tensor,
                 final_upper: torch.Tensor,
                 raw_lower: torch.Tensor,
                 raw_upper: torch.Tensor,
                 model: nn.Module,
                 X_sample: torch.Tensor,
                 y_sample: torch.Tensor,
                 p_a: float = 0.5,
                 lambda_param: float = 1e9,
                 step_size: float = 2.0,        
                 step_decay: float = 0.98):
        # Public params
        self.n_nests        = n_nests
        self.n_iterations   = n_iterations
        self.dim            = dim
        self.final_lower    = final_lower
        self.final_upper    = final_upper
        self.raw_lower      = raw_lower
        self.raw_upper      = raw_upper
        self.model          = model
        self.X              = X_sample.unsqueeze(0)   # [1, dim]
        self.y              = y_sample                # [1]
        self.p_a            = p_a
        self.lambda_param   = lambda_param
        self.step_size      = step_size               
        self.step_decay     = step_decay

        # --- population initialisation ------------------------------
        self.nests = torch.zeros((n_nests, dim), dtype=torch.float32, device=device)

        self.fitness = torch.empty(n_nests, dtype=torch.float32, device=device)
        for i in range(n_nests):
            self.fitness[i] = self._fitness(self.nests[i])

        best_idx = torch.argmin(self.fitness)
        self.best_nest    = self.nests[best_idx].clone()
        self.best_fitness = self.fitness[best_idx].item()

    # --- internals ---------------------------------------------------
    
    def _cauchy_flight(self) -> torch.Tensor:
        loc = torch.tensor(0.0, device=device)
        scale = torch.tensor(self.step_size, device=device)
        dist = torch.distributions.Cauchy(loc, scale)
        return dist.sample((self.dim,)) 

    def _new_solution(self, curr_perturb: torch.Tensor) -> torch.Tensor:
        flight = self._cauchy_flight()                              # [dim]
        cand   = clamp_tensor(curr_perturb + flight,
                              self.raw_lower, self.raw_upper)
        return cand

    def _fitness(self, perturb: torch.Tensor) -> float:
        """Fitness = ‖δ‖₂ − λ · CrossEntropy (untargeted)."""
        p_clip        = clamp_tensor(perturb, self.raw_lower, self.raw_upper)
        perturbed_raw = self.X + p_clip.unsqueeze(0)               # [1, dim]
        loss          = calculate_adversarial_loss(self.model,
                                                   perturbed_raw,
                                                   self.y).item()
        magnitude     = torch.norm(p_clip).item()
        return magnitude - self.lambda_param * loss

    # --- public API --------------------------------------------------
    def optimize(self) -> None:
        """Run Cuckoo‑Search for *n_iterations*."""
        for it in range(1, self.n_iterations + 1):
            # 1) generate new solutions
            for i in range(self.n_nests):
                cand      = self._new_solution(self.nests[i])
                cand_fit  = self._fitness(cand)

                if cand_fit < self.fitness[i]:
                    self.nests[i]  = cand
                    self.fitness[i]= cand_fit
                    if cand_fit < self.best_fitness:
                        self.best_fitness = cand_fit
                        self.best_nest    = cand.clone()

            # 2) abandon a fraction p_a of worst nests
            k = int(self.n_nests * self.p_a)
            if k > 0:
                worst_idx = torch.argsort(self.fitness)[-k:]
                for idx in worst_idx:
                    new_rand = torch.empty((self.dim,), dtype=torch.float32, device=device)\
                                  .uniform_(float(self.raw_lower),
                                            float(self.raw_upper))  
                    self.nests[idx]  = new_rand
                    self.fitness[idx]= self._fitness(new_rand)

            if it in {1, self.n_iterations} or it % 10 == 0:
                print(f"    iter {it}/{self.n_iterations} — best fitness {self.best_fitness:.5f}")

            # 3) step‑size schedule (optional)
            self.step_size *= self.step_decay                       

    def get_best_solution(self) -> torch.Tensor:
        """Return δ*  (shape [dim])."""
        return self.best_nest

# ──────────────────────────────────────────────────────────
# Main driver – sample‑by‑sample attack
# ──────────────────────────────────────────────────────────
def main() -> None:
    if model is None:
        print("Model not loaded; aborting.")
        return

    tic = time.time()
    print("Starting sample‑by‑sample adversarial generation …")

    # ---- load data --------------------------------------------------
    data_path = os.path.join("Models and Data splits", "MLP1L_500_samples_test.pkl")
    if not os.path.exists(data_path):
        print(f"Data file not found: {data_path}")
        return

    X_full_np, y_full_np = joblib.load(data_path)
    X_full = torch.tensor(X_full_np, dtype=torch.float32)
    y_full = torch.tensor(y_full_np, dtype=torch.long)
    print(f"Loaded X: {X_full.shape}, y: {y_full.shape}")

    # ---- output dirs -----------------------------------------------
    base_dir = "Generated Data/unseen_ZC-CSA_images"
    orig_dir = os.path.join(base_dir, "Original")
    adv_dir  = os.path.join(base_dir, "Adversarial")
    pert_dir = os.path.join(base_dir, "Perturbations")
    for d in (orig_dir, adv_dir, pert_dir):
        os.makedirs(d, exist_ok=True)

    # ---- hyper‑parameters ------------------------------------------
    n_nests       = 58
    n_iterations  = 100
    p_a           = 0.5
    lambda_param  = 1e10
    step_size     = 1.0      
    step_decay    = 0.98      # optional (1.0 disables decay)

    all_results = []
    processed   = 0
    adv_success = 0

    # ---- loop over digits ------------------------------------------
    for digit in range(10):     # process all digits now
        idxs = (y_full == digit).nonzero(as_tuple=True)[0]
        if idxs.numel() == 0:
            print(f"\nNo samples for digit {digit}; skipping.")
            continue

        print(f"\n── Processing digit {digit} ({idxs.numel()} samples) ──")
        X_class = X_full[idxs].to(device)
        y_class = y_full[idxs].to(device)

        for i in range(X_class.size(0)): 
            X_sample = X_class[i]                      # [dim]
            y_sample = y_class[i].unsqueeze(0)         # [1]
            gidx     = int(idxs[i])

            print(f"  sample {i+1}/{X_class.size(0)} (global {gidx})")

            # -- save original --------------------------------------
            save_uint8(X_sample.cpu().numpy().reshape(img_shape),
                       os.path.join(orig_dir, f"{gidx}_lbl{digit}.png"))

            # -- Cuckoo‑Search optimiser ---------------------------
            css = CuckooSearchSingleSample(
                n_nests      = n_nests,
                n_iterations = n_iterations,
                dim          = dim,
                final_lower  = final_lower,
                final_upper  = final_upper,
                raw_lower    = raw_lower,
                raw_upper    = raw_upper,
                model        = model,
                X_sample     = X_sample,
                y_sample     = y_sample,
                p_a          = p_a,
                lambda_param = lambda_param,
                step_size    = step_size,     
                step_decay   = step_decay,
            )
            css.optimize()
            delta_best = css.get_best_solution()

            # -- apply δ* ------------------------------------------
            adv_raw  = clamp_tensor(X_sample + delta_best, final_lower, final_upper)
            adv_norm = adv_raw.unsqueeze(0) / final_upper
            if CNN_MODE:
                adv_norm = adv_norm.view(1, 1, *img_shape)          

            with torch.no_grad():
                logits = model(adv_norm)
                pred   = torch.argmax(logits, dim=1)

            norm_l2  = torch.norm(delta_best).item()
            success  = (pred != y_sample).item()

            # -- save files ----------------------------------------
            delta_np = delta_best.cpu().numpy().reshape(img_shape)
            np.save(os.path.join(pert_dir, f"{gidx}_delta.npy"), delta_np)

            vis = ((delta_np - raw_lower_bound) / (raw_upper_bound - raw_lower_bound) * final_upper_bound)

            save_uint8(vis, os.path.join(pert_dir, f"{gidx}_delta_vis.png"))

            if success:
                adv_success += 1
                save_uint8(adv_raw.cpu().numpy().reshape(img_shape),
                           os.path.join(adv_dir,
                                        f"{gidx}_t{digit}_p{int(pred)}_m{norm_l2:.2f}.png"))

            # -- record row ----------------------------------------
            all_results.append(dict(
                digit=digit,
                index=gidx,
                true=digit,
                pred=int(pred),
                misclassified=success,
                perturb_norm_l2=norm_l2,
            ))

            processed += 1

    # ---- export CSV summary ----------------------------------------
    df = pd.DataFrame(all_results)
    csv_path = os.path.join(base_dir, "unseen_zc_csa_500x50_results.csv")
    df.to_csv(csv_path, index=False)

    # ---- per‑digit statistics --------------------------------------
    stats = (df.groupby(["digit", "misclassified"])
               .size()
               .unstack(fill_value=0)
               .rename(columns={False: "correct", True: "misclassified"}))
    
    # Ensure both columns always exist
    if "misclassified" not in stats.columns:
        stats["misclassified"] = 0
    if "correct" not in stats.columns:
        stats["correct"] = 0
    
    stats["total"]      = stats["correct"] + stats["misclassified"]
    stats["correct_%"]  = (stats["correct"]       / stats["total"] * 100).round(2)
    stats["miscls_%"]   = (stats["misclassified"] / stats["total"] * 100).round(2)


    stats_csv = os.path.join(base_dir, "unseen_zc_csa_stats_per_digit.csv")
    stats.to_csv(stats_csv)

    toc = time.time()

    # ---- pretty print ---------------------------------------------
    print("\n===== SUMMARY =====")
    print(f"Processed samples        : {processed}")
    print(f"Successful attacks       : {adv_success}")
    print(f"Detailed results  CSV    : {csv_path}")
    print(f"Per‑digit stats   CSV    : {stats_csv}")
    print(f"Total time               : {toc - tic:.2f} s\n")

    print(stats.to_string())


if __name__ == "__main__":
    main()


Model loaded successfully from Models and Data splits\MainModel_MLP1L.pt
Starting sample‑by‑sample adversarial generation …
Loaded X: torch.Size([500, 784]), y: torch.Size([500])

── Processing digit 0 (50 samples) ──
  sample 1/50 (global 0)
    iter 1/100 — best fitness -10.00000
    iter 10/100 — best fitness -2524.63713
    iter 20/100 — best fitness -714147.34456
    iter 30/100 — best fitness -18725803.68482
    iter 40/100 — best fitness -53534058.19714
    iter 50/100 — best fitness -5233780186.34167
    iter 60/100 — best fitness -14876130857.68250
    iter 70/100 — best fitness -20916379727.26514
    iter 80/100 — best fitness -21492446698.88794
    iter 90/100 — best fitness -24078899657.18372
    iter 100/100 — best fitness -30257638726.27161
  sample 2/50 (global 1)
    iter 1/100 — best fitness -10.00000
    iter 10/100 — best fitness -10.00000
    iter 20/100 — best fitness -10.00000
    iter 30/100 — best fitness -10.00000
    iter 40/100 — best fitness -10.00000
    it